[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContextLab/spam-classifier-llm-course/blob/main/Assignment2_SPAM_Classifier.ipynb)

# Assignment 2: Advanced SPAM Classifier with Multi-Method Comparison

**Course:** PSYC 51.17 - Models of Language and Communication  
**Deadline:** January 26, 2026 at 11:59 PM EST

---

## Overview

In this assignment, you will build a comprehensive spam classification system that implements multiple classification approaches, conducts rigorous comparative analysis, and performs extensive error analysis.

Please refer to the [full assignment instructions](https://contextlab.github.io/llm-course/assignments/assignment-2/) for detailed requirements and grading rubric.

---

## Table of Contents

1. [Setup and Data Loading](#setup)
2. [Part 1: Classifier Implementations](#part1)
3. [Part 2: Comprehensive Evaluation](#part2)
4. [Part 3: Error Analysis](#part3)
5. [Part 4: Adversarial Testing](#part4)
6. [Part 5: Real-World Considerations](#part5)
7. [Discussion and Reflection](#discussion)

<a id='setup'></a>
## 1. Setup and Data Loading

In [ ]:
# Install required packages
!pip install -q transformers datasets scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import os
import zipfile
import urllib.request
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

print("Setup complete!")

In [ ]:
# Download the training dataset
dataset_url = 'https://raw.githubusercontent.com/ContextLab/spam-classifier-llm-course/main/training.zip'
dataset_path = 'training.zip'

if not os.path.exists(dataset_path):
    print("Downloading dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_path)
    print("Download complete.")
else:
    print("Dataset already exists.")

In [ ]:
def load_dataset(zip_path):
    """
    Load emails from a zip archive containing spam/ and ham/ folders.
    
    Returns:
        emails: List of email texts
        labels: List of labels (1 for spam, 0 for ham)
    """
    # Extract if needed
    dataset_dir = Path(zip_path).with_suffix('')
    if not dataset_dir.exists():
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dataset_dir)
    
    emails = []
    labels = []
    
    # Load spam
    spam_folder = dataset_dir / "spam"
    for file_path in spam_folder.iterdir():
        if file_path.is_file():
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                emails.append(f.read())
                labels.append(1)
    
    # Load ham
    ham_folder = dataset_dir / "ham"
    for file_path in ham_folder.iterdir():
        if file_path.is_file():
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                emails.append(f.read())
                labels.append(0)
    
    return emails, labels

emails, labels = load_dataset('training.zip')
print(f"Loaded {len(emails)} emails")
print(f"Spam: {sum(labels)}, Ham: {len(labels) - sum(labels)}")

In [ ]:
# TODO: Split data into train/validation/test sets
from sklearn.model_selection import train_test_split

# Your code here

<a id='part1'></a>
## 2. Part 1: Multiple Classifier Implementations (40 points)

### 2.1 Traditional ML Baseline (15 points)

Implement **two** traditional ML classifiers with TF-IDF features.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# TODO: Implement TF-IDF vectorizer with appropriate parameters
# Document your feature engineering choices

vectorizer = TfidfVectorizer(
    # Your parameters here
)

# TODO: Implement first traditional classifier
# classifier1 = ...

# TODO: Implement second traditional classifier
# classifier2 = ...

### 2.2 Neural/Transformer-Based Model (15 points)

Implement a transformer-based classifier (BERT, DistilBERT, etc.).

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# TODO: Load pre-trained model and tokenizer
# model_name = 'distilbert-base-uncased'  # Recommended for speed
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# TODO: Implement fine-tuning on spam data

### 2.3 Ensemble Method (10 points)

Create an ensemble that combines your best models.

In [ ]:
# TODO: Implement ensemble method (voting, stacking, or boosting)
# Document your ensemble strategy

<a id='part2'></a>
## 3. Part 2: Comprehensive Evaluation (25 points)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

def evaluate_classifier(y_true, y_pred, y_prob=None, name="Classifier"):
    """
    Compute and display all required metrics for a classifier.
    """
    print(f"\n{'='*50}")
    print(f"{name} Evaluation")
    print('='*50)
    
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred):.4f}")
    
    if y_prob is not None:
        print(f"AUC-ROC:   {roc_auc_score(y_true, y_prob):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Ham', 'Spam']))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
    plt.title(f'{name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
# TODO: Create comparison table showing all metrics for all classifiers
# Include: Accuracy, Precision, Recall, F1, AUC-ROC

In [ ]:
# TODO: Measure computational efficiency
# Training time, inference time, model size, throughput
import time

In [ ]:
# TODO: Cross-validation with statistical significance testing
from sklearn.model_selection import cross_val_score

<a id='part3'></a>
## 4. Part 3: Error Analysis (20 points)

In [ ]:
# TODO: Identify and analyze at least 20 misclassified emails
# - 10 false positives (ham classified as spam)
# - 10 false negatives (spam classified as ham)

# Categorize errors and analyze patterns

In [ ]:
# TODO: Feature importance analysis
# What words/patterns are most predictive?

<a id='part4'></a>
## 5. Part 4: Adversarial Testing (10 points)

In [ ]:
# TODO: Create at least 5 adversarial emails
# Test on all classifiers and analyze which are most robust

adversarial_emails = [
    # Example: spam trying to evade detection
    """Fr33 V1agra! Click here for amazing deals!
    
    The weather today is quite nice. I hope you are doing well.
    Best regards from a legitimate sender.""",
    
    # Add more adversarial examples
]

In [ ]:
# TODO: Test robustness against perturbations
# - Typos and misspellings
# - Case variations
# - Synonym replacement

<a id='part5'></a>
## 6. Part 5: Real-World Considerations (5 points)

### Class Imbalance Discussion

(How did you handle class imbalance? What happens if spam/ham ratio changes in production?)

### Deployment Scenarios

Which model would you choose for:
- **Mobile email app** (fast inference, small size): 
- **Email server** (high throughput): 
- **Maximum accuracy** (no constraints): 

(Justify with evidence from your experiments)

<a id='discussion'></a>
## 7. Discussion and Reflection

### Key Findings

(What did you learn about spam classification? Which approach worked best and why?)

### Limitations

(What are the limitations of your approach? What would you improve with more time?)

### Reflection

(What was challenging? What surprised you about the results?)

---

## Submission Checklist

- [ ] At least 3 classifiers implemented (2 traditional + 1 neural + ensemble)
- [ ] All metrics computed and comparison table created
- [ ] Cross-validation with statistical testing
- [ ] At least 20 error cases analyzed
- [ ] At least 5 adversarial examples created and tested
- [ ] Deployment recommendations with justification
- [ ] Discussion and reflection complete
- [ ] Notebook runs without errors

**To submit:** Commit and push this notebook to your GitHub repository before the deadline.